## Presentation Types with Hidden Types
## Also Network Visualization with Louvain Communities

* March 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
    ALSO
    
    * Finds "hidden" types within a longer Fuga.  That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above.

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [2]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0001.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei
{'title': 'Veni speciosam', 'composer': 'Lupi, Johannes'}


In [47]:
def class_test(piece, nr, mel_ng, entries, edit_distance_threshold, include_hidden_types):

    """This function uses several other functions to classify the entries in a given piece.
    The output is a list, in order of offset, of each presentation type, including information about
    measures/beats
    starting offset
    soggetti involved
    melodic intervals of entry
    time intervals of entry

    set the length of the soggetti with `melodic_ngram_length`
    set the maximum difference between similar soggetti with `edit_distance_threshold`
    for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
    to include all the hidden PENs and IDS (those found within longer Fugas),
    use `include_hidden_types == True`.
    For faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`


    """
    # Classifier with Functions
    points = pd.DataFrame(columns=['Composer',
                 'Title',
                 'First_Offset',
                 'Measures_Beats',
                 'Melodic_Entry_Intervals',
                 'Offsets',
                 'Soggetti',
                 'Time_Entry_Intervals',
                 'Voices',
                 'Presentation_Type'])
    points2 = pd.DataFrame()
    
    # new_offset_list = []
#     nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # The following are now all set in the Notebook as argument
    # durations and ngrams of durations
    # dur = piece.getDuration(df=nr)
    # dur_ng = piece.getNgrams(df=dur, n=3)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    #mel_ng = piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mels_stacked = entries.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(entries)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    distance_factor = edit_distance_threshold + 1
    filtered_dist_stack = dist_stack[dist_stack[0] < distance_factor]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    if include_hidden_types == False:

        for matches in full_list_of_matches["match"]:
            related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
            entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
            offset_list = entry_array.index.to_list()
            split_list = list(split_by_threshold(offset_list))
            for item in split_list:

                # print(item)

                offset_gap_limit = 40.0
            
                
            # here is the list of starting offsets of the original set of entries:  slist
# 
                temp = temp_dict_of_details(item, entry_array, det, matches, piece)
                a = temp["Time_Entry_Intervals"] 
                if (all(x < offset_gap_limit for x in a)):

                    



                    points = points.append(temp, ignore_index=True)
                    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
                    # points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
                    points = points[points['Offsets'].apply(len) > 1]

        points["Offsets_Key"] = points["Offsets"].apply(offset_joiner)
        points.drop_duplicates(subset=["Offsets_Key"], keep='first', inplace = True)
        points['Flexed_Entries'] = points["Soggetti"].apply(len) > 1
        points["Number_Entries"] = points["Offsets"].apply(len)
        col_order = ['Composer',
                 'Title',
                 'First_Offset',
                 'Measures_Beats',
                 'Melodic_Entry_Intervals',
                 'Offsets',
                 'Soggetti',
                 'Time_Entry_Intervals',
                 'Voices',
                 'Presentation_Type',
                  'Number_Entries',
                'Flexed_Entries']
        points = points.reindex(columns=col_order).sort_values("First_Offset").reset_index(drop=True)
        # return points

    elif include_hidden_types == True:
        hidden_types_list = ["PEN", "ID"]

        for matches in full_list_of_matches["match"]:
            related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
            entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
            offset_list = entry_array.index.to_list()
            split_list = list(split_by_threshold(offset_list))
            for item in split_list:
            # here is the list of starting offsets of the original set of entries:  slist

                offset_gap_limit = 40.0
                
                temp = temp_dict_of_details(item, entry_array, det, matches, piece)

                b = temp["Time_Entry_Intervals"] 
                if (all(x < offset_gap_limit for x in b)):


                    points = points.append(temp, ignore_index=True)
                    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
                    # points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
                    points = points[points['Offsets'].apply(len) > 1]
        # return(points)

        
            for item in split_list:
    # #         # here is the list of starting offsets of the original set of entries:  slist
                offset_gap_limit = 40.0

                temp = temp_dict_of_details(item, entry_array, det, matches, piece)

                c = temp["Time_Entry_Intervals"] 
                if (all(x < offset_gap_limit for x in c)):

                    lto = len(temp["Offsets"])
                    if lto > 2 :
                        for r in range(3, 6):
                            list_combinations = list(combinations(item, r))
                            for slist in list_combinations:

                                temp = temp_dict_of_details(slist, entry_array, det, matches, piece)

                                temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])
                                # print(temp)

                                if 'PEN' in temp["Presentation_Type"]:
                                    points2 = points2.append(temp, ignore_index=True)
                                if 'ID' in temp["Presentation_Type"]:
                                    points2 = points2.append(temp, ignore_index=True)

        
        

        points2 = points2[points2["Presentation_Type"].isin(hidden_types_list)]
        # return(points2)

        
        points_combined = points.append(points2, ignore_index=True)
        points_combined["Offsets_Key"] = points_combined["Offsets"].apply(offset_joiner)
        points_combined.drop_duplicates(subset=["Offsets_Key"], keep='first', inplace = True)
        points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
        points_combined["Number_Entries"] = points_combined["Offsets"].apply(len)
        col_order = ['Composer',
                 'Title',
                 'First_Offset',
                 'Measures_Beats',
                 'Melodic_Entry_Intervals',
                 'Offsets',
                 'Soggetti',
                 'Time_Entry_Intervals',
                 'Voices',
                 'Presentation_Type',
                  'Number_Entries',
                'Flexed_Entries']
        # points_combined = points_combined.sort_values("First_Offset").reset_index(drop=True)
        points_combined = points_combined.reindex(columns=col_order).sort_values("First_Offset").reset_index(drop=True)
        return points_combined
        
    

                        


## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



In [44]:
mylist = [2.0, 6.0, 10.0]
offset_gap_limit = 20.0
if (all(x < offset_gap_limit for x in mylist)):
    print('yes')

yes


In [48]:
include_hidden_types = True
combine_unisons = True
melodic_ngram_length = 4
edit_distance_threshold = 1
nr = piece.getNoteRest(combineUnisons=combine_unisons)
dur = piece.getDuration(df=nr)
mel = piece.getMelodic(df=nr, kind='d', end=False)
mel_ng = piece.getNgrams(df=mel, n=melodic_ngram_length)
entries = piece.getEntries(mel_ng)
output = class_test(piece, nr, mel_ng, entries, edit_distance_threshold, include_hidden_types)
# dur.to_csv('test.csv')

#### Below Find Source Code and Explanation of the Method

In [49]:
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Lupi, Johannes",Veni speciosam,0.0,"[1/1.0, 2/3.0, 5/1.0, 6/3.0, 9/2.0, 10/1.0]","[P-4, P-5, P-4, P4, P8]","[0.0, 12.0, 32.0, 44.0, 66.0, 72.0]","[(5, -2, 2, 3), (4, -2, 2, 3)]","[12.0, 20.0, 12.0, 22.0, 6.0]","[Superius, Contratenor, PrimusTenor, Bassus, S...",FUGA,6,True
1,"Lupi, Johannes",Veni speciosam,0.0,"[1/1.0, 2/3.0, 5/1.0, 6/3.0]","[P-4, P-5, P-4]","[0.0, 12.0, 32.0, 44.0]","[(5, -2, 2, 3), (4, -2, 2, 3)]","[12.0, 20.0, 12.0]","[Superius, Contratenor, PrimusTenor, Bassus]",ID,4,True
2,"Lupi, Johannes",Veni speciosam,94.0,"[12/4.0, 16/4.0]",[P4],"[94.0, 126.0]","[(-3, 2, 2, -2)]",[32.0],"[Bassus, Bassus]",FUGA,2,False
3,"Lupi, Johannes",Veni speciosam,138.0,"[18/2.0, 21/2.0, 24/2.0]","[M9, M-9]","[138.0, 162.0, 186.0]","[(-2, -3, 2, 2), (-2, -3, 3, 2)]","[24.0, 24.0]","[PrimusTenor, Superius, PrimusTenor]",PEN,3,True
4,"Lupi, Johannes",Veni speciosam,138.0,"[18/2.0, 18/4.0, 20/4.0, 21/2.0]","[P-4, P8, P5]","[138.0, 142.0, 158.0, 162.0]","[(-2, -3, 2, 2), (-2, -3, 3, 2)]","[4.0, 16.0, 4.0]","[PrimusTenor, Bassus, Contratenor, Superius]",ID,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...
106,"Lupi, Johannes",Veni speciosam,978.0,"[122/2.0, 122/4.0, 123/2.0]","[P1, P8]","[978.0, 982.0, 986.0]","[(2, -2, -3, 2)]","[4.0, 4.0]","[SecundusTenor, PrimusTenor, Superius]",PEN,3,False
107,"Lupi, Johannes",Veni speciosam,978.0,"[122/2.0, 122/4.0, 123/2.0, 125/1.0, 126/4.0, ...","[P1, P8, P-5, P-5, P-8, P8, P1]","[978.0, 982.0, 986.0, 1000.0, 1014.0, 1020.0, ...","[(2, -2, -3, 2)]","[4.0, 4.0, 14.0, 14.0, 6.0, 10.0, 8.0]","[SecundusTenor, PrimusTenor, Superius, Contrat...",FUGA,8,False
108,"Lupi, Johannes",Veni speciosam,978.0,"[122/2.0, 123/2.0, 128/4.0, 129/4.0]","[P8, M-9, P1]","[978.0, 986.0, 1030.0, 1038.0]","[(2, -2, -3, 2)]","[8.0, 44.0, 8.0]","[SecundusTenor, Superius, Contratenor, Secundu...",ID,4,False
109,"Lupi, Johannes",Veni speciosam,982.0,"[122/4.0, 125/1.0, 127/3.0, 129/4.0]","[P4, P-12, P8]","[982.0, 1000.0, 1020.0, 1038.0]","[(2, -2, -3, 2)]","[18.0, 20.0, 18.0]","[PrimusTenor, Contratenor, Bassus, SecundusTenor]",ID,4,False


### Run Classifier on Several Pieces at Once

Results are combined into a single dataframe

In [ ]:
output["Presentation_Type"].value_counts()

In [ ]:


git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')
piece_list =  ['CRIM_Mass_0005_3.mei',
             'CRIM_Mass_0005_4.mei',
             'CRIM_Mass_0005_5.mei',
             'CRIM_Model_0001.mei',
             'CRIM_Mass_0002_1.mei',
             'CRIM_Mass_0002_2.mei',
             'CRIM_Mass_0002_3.mei',
             'CRIM_Mass_0002_4.mei',
             'CRIM_Mass_0002_5.mei',
             'CRIM_Model_0015.mei',
             'CRIM_Mass_0013_1.mei',
             'CRIM_Mass_0013_2.mei',
             'CRIM_Mass_0013_3.mei',
             'CRIM_Mass_0013_4.mei',
             'CRIM_Mass_0013_5.mei',
             'CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']


In [ ]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

piece_list = ['CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']

In [ ]:
include_hidden_types = False
melodic_ngram_length = 4
edit_distance_threshold = 0
final = pd.DataFrame()
for work in piece_list:
    url = git_prefix + work
    piece = importScore(url)   
    nr = piece.getNoteRest(combineUnisons=True)
    dur = piece.getDuration(df=nr)
    mel = piece.getMelodic(df=nr)
    mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
    output = classify_entries_as_presentation_types(piece, nr, mel_ng, edit_distance_threshold, include_hidden_types)
    final = final.append(output, ignore_index=True)
final

In [ ]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["ALL_SOG"] = final["MINT"] + '_' + final["SOG"]
final

In [ ]:
filtered = final.loc[final['Number_Entries'] < 20] 
filtered

### Network Visualization with Louvain Communities

In [ ]:

f2 = filtered.groupby('MINT')['Title'].apply(list).reset_index()
f2


In [ ]:
pairs = f2.Title.apply(lambda x: list(combinations(x, 2)))
pairs

In [ ]:
pairs2 = pairs.explode().dropna()
pairs2

In [ ]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

In [ ]:
pd.Series(unique_pairs).isna().sum()

In [ ]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [ ]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [ ]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [ ]:
pyvis_graph.from_nx(G)

In [ ]:
pyvis_graph.show('MINT.html')


#### Below is Development Work

In [ ]:
filtered = output.loc[output['Number_Entries'] < 4] 
filtered

In [ ]:
output = output.loc[output['Presentation_Type'] == "PEN"] 
output

In [ ]:
offset_diffs = [2.0, 1.0, 2.0, 3.0, 5.0, 6.0]
# some_list[start:stop:step]
alt_list = offset_diffs[::2]
alt_list

In [ ]:
# this works with ONE list of offsets

points2 = pd.DataFrame()
split_list = [90.0, 94.0, 102.0, 106.0, 134.0, 146.0, 162.0]

l = len(split_list)  
for r in range(3, l):
    list_combinations = list(combinations(split_list, r))
#             combo_time_ints = []
    for combo in list_combinations:
        combo_time_ints = numpy.diff(combo).tolist()
        combo_array = entry_array[entry_array.index.get_level_values(0).isin(combo)]
        combo_voice_list = combo_array['voice'].to_list()
        combo_patterns = combo_array['pattern']
        unique_combo_patterns = list(set(combo_patterns))
        tone_coordinates =  list(zip(combo, combo_voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        hidden_type = classify_by_offset(combo_time_ints)

        meas_beat = det[det.index.get_level_values('Offset').isin(combo)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()

        combo_temp = {'First_Offset': combo[0], 
            'Offsets': combo, 
            'Measures_Beats': meas_beat_list,
            'Presentation_Type': hidden_type,
            "Soggetti": unique_combo_patterns,
            'Voices': combo_voice_list, 
            'Time_Entry_Intervals': combo_time_ints, 
            'Melodic_Entry_Intervals': mel_ints}

        if 'PEN' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        if 'ID' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        
        
# combo_time_ints
# combo_array
# # combo_voice_list
# # combo_patterns
# # unique_combo_patterns
# # tone_coordinates
# # mel_ints
# # combo_temp
points2

In [ ]:
# this finds hidden fugas.  
# try to run each of the first set of results above ('points') through this tool, then append the 
# new results to the full DF, and sort again.  
# mark each long pattern with 'has hidden pattern' boolean?  or ?

sample_list = points["Offsets"][4]

hidden_pts = []
n = len(sample_list)
for item in range(3, n):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        

list_combinations

In [ ]:
def classify_entries_as_presentation_types(piece):
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points["Offsets"].apply(len)     
    return points2


In [ ]:
# This test works

points = pd.DataFrame()
points2 = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=4)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=4)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    # here is the list of starting offsets of the original set of entries:  slist
    slist = split_list[0]
    temp = temp_dict_of_details(slist, entry_array, det, matches)

    points = points.append(temp, ignore_index=True)
    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
    points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
    points = points[points['Offsets'].apply(len) > 1]

    test = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]

    l = len(test)  
    for item in range(3, l):
        list_combinations = list(combinations(test, item))
        for group in list_combinations:
            group_time_ints = numpy.diff(group).tolist()
            hidden_type = classify_by_offset(group_time_ints)
            for item in group:
    #         print(item)
                array = group[entry_array.index.get_level_values(0).isin(item)]
                short_offset_list = array.index.to_list()
                time_ints = numpy.diff(array.index).tolist()
                voice_list = array['voice'].to_list()
                if 'PEN' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
                if 'ID' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
# len(split_list[0])           

In [ ]:
#  This shows how the classifier works:

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

In [ ]:
# This shows how combinations works for a given set of time intervals
offset_diffs = [12.0, 32.0, 12.0, 4.0]
l = len(offset_diffs)
# print(l)
if l > 2:
    for r in range(3, l):
        print(r)
        list_combinations = list(combinations(offset_diffs, r))
#         for slist in list_combinations:
        print(list_combinations)

In [ ]:
slist = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]
l = len(slist)
# for r in range(3, 6):
list_combinations = list(combinations(slist, 4))
#     for tiny_list in list_combinations:

In [ ]:
print(list_combinations)

In [ ]:
list_offsets = [294.0, 298.0, 306.0, 310.0]

In [ ]:
offset_diffs = [4, 5, 6]

In [ ]:
alt_list = offset_diffs[::2]

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

In [ ]:
def getDistance2(self, df=None, n=3):
    if df is None:
            df = self.getMelodic('z', True, True)
            df = self.getNgrams(df=df, n=n, exclude=['Rest'])
    uni = df.stack().unique()
    ser = pd.Series(uni)
    df = pd.DataFrame.from_records(ser.apply(lambda cell: tuple(int(i) for i in cell.split(', '))))
    cols = [(df - df.loc[i]).abs().apply(sum, axis=1) for i in df.index]
    dist = pd.concat(cols, axis=1)
    dist.columns = uni
    dist.index = uni
    return dist

In [ ]:
piece

In [ ]:
melodic_ngram_length = 4
edit_distance_threshold = 1
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
mel = piece.getMelodic(df=nr, kind='d', end=False)
mel_ng = piece.getNgrams(df=mel, n=melodic_ngram_length)
mel_ng

In [ ]:
a = importedPiece.getDistance2(df=mel_ng)